In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
import pandas as pd
import unicodedata

## Extração dos dados

In [59]:


#Criando um dicionário com todos os caminhos dos arquivos. Isso vai ser bom porque utilizarei as chaves para criar uma coluna de "Ano"
arquivos = {
    '2022': 'data/156_2022.csv', #utilizamos a base de 2022 pois a de 2023 estava dando erro na hora de juntar as tabelas, todas as colunas estavam ficando juntas em apenas umna
}

lista_dataframes = [] #Para armazenar todos os dataframes

for ano, arquivo in arquivos.items():
    try:
        if ano == '2022':
            df = pd.read_csv(arquivo,sep=';',encoding='utf-8') # O arquivo do ano de 2022 esta em "UTF-8" se se usassemos "latin1" os caracteres "Ç" "~" iriam ficar errados
        else:
            df = pd.read_csv(arquivo,sep=';',encoding='latin1')
        
        df['_id'] = range(1, len(df) + 1) #Criação do campo id pois os arquivos csv vieram sem

        cols = ['_id'] + [c for c in df.columns if c != '_id']
        df = df[cols]
        
        df['ano_origem'] = ano #criando uma nova coluna para salvar o ano dos chamados

        lista_dataframes.append(df)
    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")


df = pd.concat(lista_dataframes, ignore_index=True)


In [60]:
#Verificando o resultado final e as informações sobre os tipos de dados
display(df.head(5))
display(df.info())

,_id,GRUPOSERVICO_CODIGO,GRUPOSERVICO_DESCRICAO,SERVICO_CODIGO,SERVICO_DESCRICAO,LOGRADOURO,NUMERO,BAIRRO,RPA,DATA_DEMANDA,SITUACAO,DATA_ULT_SITUACAO,latitude,longitude,ano_origem
0,1,10,DENUNCIAS,700,DENUNCIA ARBOVIROSE ...,10t prazeres,,AREIAS,5,2022-06-20,PREPARAÇÃO,2022-09-22,-8.0981629,-34.9249594,2022
1,2,7,ARBORIZAÇÃO,16,VISTORIA DE ARVORES ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-07-27,ATENDIDA,2022-09-08,-8.0028869,-34.9274109,2022
2,3,7,ARBORIZAÇÃO,16,VISTORIA DE ARVORES ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-10-16,PREPARAÇÃO,2022-10-17,-8.0028869,-34.9274109,2022
3,4,12,ESCADARIA,57,RECUP. DE ESCADARIA ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-02-16,CADASTRADA,2022-02-16,-8.0028869,-34.9274109,2022
4,5,12,ESCADARIA,57,RECUP. DE ESCADARIA ...,1sb do sargento,,NOVA DESCOBERTA,3,2022-06-02,CADASTRADA,2022-06-02,,,2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96633 entries, 0 to 96632
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   _id                     96633 non-null  int64 
 1   GRUPOSERVICO_CODIGO     96633 non-null  int64 
 2   GRUPOSERVICO_DESCRICAO  96633 non-null  object
 3   SERVICO_CODIGO          96633 non-null  int64 
 4   SERVICO_DESCRICAO       96633 non-null  object
 5   LOGRADOURO              96633 non-null  object
 6   NUMERO                  96633 non-null  object
 7   BAIRRO                  96633 non-null  object
 8   RPA                     96633 non-null  int64 
 9   DATA_DEMANDA            96633 non-null  object
 10  SITUACAO                96633 non-null  object
 11  DATA_ULT_SITUACAO       96633 non-null  object
 12  latitude                96633 non-null  object
 13  longitude               96633 non-null  object
 14  ano_origem              96633 non-null  object
dtypes:

None

In [61]:
colunas_de_texto = df.select_dtypes(include=['object']).columns
for col in colunas_de_texto:
    df[col] = df[col].apply(
        lambda x: unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8').strip() 
        if pd.notna(x) else x)
    df[col] = df[col].str.upper()
    

In [62]:
#Verificando o resultado final e as informações sobre os tipos de dados
display(df.head(5))
display(df.info())

,_id,GRUPOSERVICO_CODIGO,GRUPOSERVICO_DESCRICAO,SERVICO_CODIGO,SERVICO_DESCRICAO,LOGRADOURO,NUMERO,BAIRRO,RPA,DATA_DEMANDA,SITUACAO,DATA_ULT_SITUACAO,latitude,longitude,ano_origem
0,1,10,DENUNCIAS,700,DENUNCIA ARBOVIROSE,10T PRAZERES,,AREIAS,5,2022-06-20,PREPARACAO,2022-09-22,-8.0981629,-34.9249594,2022
1,2,7,ARBORIZACAO,16,VISTORIA DE ARVORES,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-07-27,ATENDIDA,2022-09-08,-8.0028869,-34.9274109,2022
2,3,7,ARBORIZACAO,16,VISTORIA DE ARVORES,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-10-16,PREPARACAO,2022-10-17,-8.0028869,-34.9274109,2022
3,4,12,ESCADARIA,57,RECUP. DE ESCADARIA,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-02-16,CADASTRADA,2022-02-16,-8.0028869,-34.9274109,2022
4,5,12,ESCADARIA,57,RECUP. DE ESCADARIA,1SB DO SARGENTO,,NOVA DESCOBERTA,3,2022-06-02,CADASTRADA,2022-06-02,,,2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96633 entries, 0 to 96632
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   _id                     96633 non-null  int64 
 1   GRUPOSERVICO_CODIGO     96633 non-null  int64 
 2   GRUPOSERVICO_DESCRICAO  96633 non-null  object
 3   SERVICO_CODIGO          96633 non-null  int64 
 4   SERVICO_DESCRICAO       96633 non-null  object
 5   LOGRADOURO              96633 non-null  object
 6   NUMERO                  96633 non-null  object
 7   BAIRRO                  96633 non-null  object
 8   RPA                     96633 non-null  int64 
 9   DATA_DEMANDA            96633 non-null  object
 10  SITUACAO                96633 non-null  object
 11  DATA_ULT_SITUACAO       96633 non-null  object
 12  latitude                96633 non-null  object
 13  longitude               96633 non-null  object
 14  ano_origem              96633 non-null  object
dtypes:

None

## Verificando valores únicos nas colunas

In [27]:
df.nunique(dropna=False)

GRUPOSERVICO_CODIGO          33
GRUPOSERVICO_DESCRICAO       46
SERVICO_CODIGO              176
SERVICO_DESCRICAO           205
LOGRADOURO                16006
NUMERO                    11623
BAIRRO                      190
RPA                           6
DATA_DEMANDA               3469
SITUACAO                     12
DATA_ULT_SITUACAO          1007
latitude                  68516
longitude                 64953
ano_origem                    3
dtype: int64

## Removendo Valores Duplicados

In [36]:
df_duplicados = df.copy()
df_duplicados = df_duplicados.drop('_id', axis=1)
print("Linhas duplicadas: ", df_duplicados.duplicated().sum())

Linhas duplicadas:  1
